## Golden wind in r/WallStreetBets

### Using Pushshift API to fetch data

The PSAW library (different from the `praw` library!) lets you access this data resource as well: [PushShift.io API Wrapper](https://github.com/dmarx/psaw)

* r/WallStreetBets Research Project Phase 3
* Team 4
* Anqi Fang, Jiashuo Sun, Raymond Su

**This is a note book for part 2 of our analysis - Data preparation.**


In [1]:
from psaw import PushshiftAPI

api = PushshiftAPI()
# Define some keys for submission attribtues
filter_keys = ['author','id','num_comments',
               'score','upvote_ratio','title','selftext']

In [2]:
# Handling dates and times
from datetime import datetime
#also import pandas
import pandas as pd

In [3]:
# Define the starting and ending time frame

start = int(datetime(2020, 12, 8).timestamp())
end = int(datetime(2021, 3, 1).timestamp())

search = api.search_submissions(after=start,
                                before = end,
                                subreddit='wallstreetbets',
                                filter=filter_keys,
                                sort='asc',
                                limit=None)

In [ ]:
# Storage for the results
all_subs = []

# Loop through the search results to actually get data
for i,sub in enumerate(search):
    
    # Add each result's dictionary (the .d_ attribute) to the all_subs
    all_subs.append(sub.d_)
    
    # Print out status updates every 10,000 submissions
    if i % 10000 == 0:
        
        # The current time so you know how long in between updates
        time_now = datetime.now().time().replace(microsecond=0)
        
        # The date of the submission to give you an idea of how far along you are
        record_date = datetime.utcfromtimestamp(sub.d_['created']).date()
        
        # Print it out
        print("{0:,} for {1} received at {2}".format(i,record_date,time_now))

In [5]:
# turn list of submission into DataFrame
subs_df = pd.DataFrame(all_subs)
print('{:,}'.format(len(all_subs)))

# translate Unix time to understandable format
subs_df['timestamp'] = subs_df['created'].apply(datetime.utcfromtimestamp)
subs_df['date'] = subs_df['timestamp'].apply(lambda x:x.date())

# save data as csv file
subs_df.to_csv('all_submissions.csv',encoding='utf8',index=False)

subs_df.head()

837,282


,author,created_utc,id,num_comments,score,selftext,title,upvote_ratio,created,timestamp,date
0,Zckary,1607414431,k90ai4,3,1,19 year old college student in debt looking fo...,SERIOUS INQUIRIES ONLY,1.0,1.607440e+09,2020-12-08 15:00:31,2020-12-08
1,virginityisgood,1607414453,k90ao2,46,1,,"Money isn’t real, will make everything back th...",1.0,1.607440e+09,2020-12-08 15:00:53,2020-12-08
2,shogunz888,1607414850,k90dkb,17,1,,TSLA to the moon! 🚀🚀🚀🚀🚀,1.0,1.607440e+09,2020-12-08 15:07:30,2020-12-08
3,BelmontMan,1607415309,k90gqc,0,1,[removed],Investing or gambling?,1.0,1.607441e+09,2020-12-08 15:15:09,2020-12-08
4,yaplugxbl,1607415449,k90hmz,0,1,[removed],SNDL?,1.0,1.607441e+09,2020-12-08 15:17:29,2020-12-08


### Data Cleaning

In [ ]:
import nltk, re, json, io
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
### check the missing values in all columns
subs_df.isnull().sum()

In [ ]:
### drop the missing values
subs_df = subs_df[subs_df.selftext.notnull()]

As I checked random rows in this dataset, I found some rows containing "removed" "deleted" as the text value of the reddit submission. So I added a filter to drop those rows.

In [ ]:
### remove the rows with "removed" as the text value
subs_df = subs_df[subs_df.selftext != "[removed]"]
subs_df = subs_df[subs_df.selftext != "[deleted]"]

In [ ]:
text_list = subs_df['selftext'].tolist()

In [ ]:
### clean the text
def textcleaner(row):
    # convert every word to lower case
    row = row.lower()
    #remove urls
    row  = re.sub(r'http\S+', '', row)
    #remove mentions
    row = re.sub(r"(?<![@\w])@(\w{1,25})", '', row)
    #remove hashtags
    row = re.sub(r"(?<![#\w])#(\w{1,25})", '',row)
    #remove other special characters
    row = re.sub('[^A-Za-z .-]+', '', row)
    ### remove the "." and "-"
    row  = re.sub('[.-]', '', row)
    #remove digits
    row = re.sub('\d+', '', row)
    row = row.strip(" ")
    row = re.sub('\s+', ' ', row)
    return row

In [ ]:
cleaned_textlist = []
### clean the text in the list
for t in text_list:
    cleaned_textlist.append(textcleaner(t))

Besides the general text cleaning, we also want to remove the stopwords, which do not carry contextual meanings in themselves.

In [ ]:
### remove stopwords
stopeng = set(stopwords.words('english'))

cleaned = []
for row in cleaned_textlist:
    tokens = word_tokenize(row)
    tokens_nostop = [w for w in tokens if w not in stopeng]
    cleaned_text = " ".join(tokens_nostop)
    cleaned.append(cleaned_text)

In [ ]:
subs_df['cleaned_text'] = cleaned

The next step is to tokenize the text, I choose the function word_tokenize() because the text has already been processed and cleaned.

In [ ]:
### tokenize the text
token_result = []

for item in cleaned:
    tokened = word_tokenize(item)
    token_result.append(tokened)
subs_df['tokened_text'] = token_result

In [ ]:
### save the results as a csv file
subs_df.to_csv("PATH/cleaned_submissions.csv", index = False)